In [1]:
%pip install openai
%pip install chess


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.
Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [20]:
# import required libraries
import os
import openai
import json
import numpy as np
import pandas as pd
import chess
import copy
import time
import datetime


from ast import literal_eval
from dotenv import load_dotenv, find_dotenv
from sklearn.model_selection import train_test_split


In [3]:
# create dataframe from downloaded dataset
df = pd.read_csv('data/db.csv')
# create dataframe of one move chess puzzles
df_one = df[df['Themes'].str.contains('mateIn1')].reset_index(drop=True)


In [25]:
# import environment variable into notebook
_ = load_dotenv(find_dotenv())  # read local .env file
openai.api_key = os.getenv('OPENAI_API_KEY_UPV')

# list openai available models given the api key.
ids = [item["id"] for item in (openai.Model.list())['data']]
print(ids)


['babbage', 'text-davinci-003', 'davinci', 'text-davinci-edit-001', 'babbage-code-search-code', 'text-similarity-babbage-001', 'code-davinci-edit-001', 'ada', 'babbage-code-search-text', 'babbage-similarity', 'gpt-3.5-turbo-16k-0613', 'code-search-babbage-text-001', 'text-curie-001', 'gpt-3.5-turbo-0301', 'gpt-3.5-turbo-16k', 'code-search-babbage-code-001', 'text-ada-001', 'text-similarity-ada-001', 'text-davinci-002', 'curie-instruct-beta', 'ada-code-search-code', 'ada-similarity', 'code-search-ada-text-001', 'text-search-ada-query-001', 'davinci-search-document', 'ada-code-search-text', 'text-search-ada-doc-001', 'davinci-instruct-beta', 'text-similarity-curie-001', 'code-search-ada-code-001', 'ada-search-query', 'text-search-davinci-query-001', 'curie-search-query', 'davinci-search-query', 'babbage-search-document', 'ada-search-document', 'text-search-curie-query-001', 'text-babbage-001', 'text-search-babbage-doc-001', 'whisper-1', 'curie-search-document', 'text-davinci-001', 'text-

In [5]:
def push_move(board: chess.Board, move: chess.Move):
    """ 
    args:
        board (chess.Board)
        moves (chess.Move)
    """
    if board.is_legal(move):
        board.push(move)
    return board


def is_legal_move(board: chess.Board, move: chess.Move) -> bool:
    """ 
    Function that given a fen an a move checks
    if the provided move is legal
    args:
        board (chess.Board)
        moves (str)
    returns:
        bool  
    """
    return board.is_legal(move)


In [6]:
def build_filename(template, temperature, dir, model, ext="txt"):
    """
        generate filename name, if it exists, it appends an index number
        at the end of it.
    """
    current_date = datetime.datetime.now().strftime("%d_%m")
    filename = f"{current_date}-{template}-{model}-{temperature}.{ext}"
    full_path = os.path.join(dir, filename)

    index = 1
    while os.path.exists(full_path):
        new_filename = f"{current_date}-{template}-{model}-{temperature}_{index}.{ext}"
        full_path = os.path.join(dir, new_filename)
        index += 1

    return full_path


def build_columns(variability=3):
    columns = []

    columns.append(f"puzzleId")
    columns.append(f"datasetMove")

    for i in range(variability):
        columns.append(f"gpt{i}-move")
        columns.append(f"gpt{i}-valid")
        columns.append(f"gpt{i}-done")

    columns.append(f"gpt-results")
    return columns


In [7]:
# build dataframe that with the provided moves, the puzzle is completed
# should be a redundant code as the dataset should be all puzzles completed
# but just in case of some puzzle stored not done correctly.

checkmate_df = pd.DataFrame(columns=df.columns)

for index, row in df_one.iterrows():
    board = chess.Board(row['FEN'])
    chess_moves = [chess.Move.from_uci(move) for move in row['Moves'].split()]

    for move in chess_moves:
        if (move in board.legal_moves):
            push_move(board=board, move=move)

    if (board.is_checkmate()):
        checkmate_df = pd.concat([checkmate_df, row.to_frame().T])
    else:
        print(row['PuzzleId'])


In [34]:
# global constant variables

# output dir to save data files
OUTPUT_DIR = "./out"
# name of files to append to filename
TEMPLATE_FILE_PATH = "data"
# type of extension for data files
EXTENSION_FILE = "csv"

# error files dir path
OUTPUT_ERROR_DIR = "./out/error"
# name of error files to append to error filename
TEMPLATE_ERROR_FILE_PATH = "error"

# openai variables

TEMPERATURE = 0
# AVAILABLE MODELS [gpt-3.5-turbo-16k, gpt-4, gpt-4-0613]
MODEL = 'gpt-3.5-turbo-16k'

# config variables

# increase time in case of error *`ServiceUnavailableError: The server is overloaded or not ready yet.`*
REQ_INTERVAL = 5  # 10 sec
VARIABILITY_TRIES = 3
MAX_PUZZLES = 5

# debug flag
DEBUG = True


In [9]:
def request(
    prompt="",
    context=np.array([]),
):
    """
    Helper function to make a request to openai api, and return it's response.
    args:
        prompt (string)
        model (str)
        temperature (int)
    returns:
        str
    """
    context = np.append(
        context,
        np.array([{
            "role": 'user',
            "content": f"""
                {prompt}
            """,
        }]), axis=0)

    response = openai.ChatCompletion.create(
        model=MODEL,
        messages=context.tolist(),
        temperature=TEMPERATURE,  # this is the degree of randomness of the model's output
    )

    return response.choices[0].message["content"]


In [10]:
def process_test_row(row, context, temperature=0, debug=False):
    # create chess board with 'fen' dataset
    board = chess.Board(row['FEN'])
    # create chess moves from 'uci' moves from dataset
    chess_moves = [chess.Move.from_uci(move) for move in row['Moves'].split()]
    # retrieve last move
    last_move = chess_moves.pop()

    # add all moves except the last one.
    for move in chess_moves:
        push_move(board=board, move=move)

    # generate prompt
    prompt = f"""
        "fen": {board.fen()}
        "valid_moves": { json.dumps([move.uci() for move in list(board.legal_moves)])}
    """

    # req to gpt model
    req = request(prompt=prompt, context=context).replace(" ", "")

    if (debug):
        print(f"[gpt-response]: {req}")

    res = literal_eval(req)
    # check if valid gpt res move
    valid_res = is_legal_move(
        board=board, move=chess.Move.from_uci(res['move']))

    # info printings
    if (debug):
        print(f"[gpt-move | dataset-move] : {res['move']} | {last_move}")

    if (valid_res is not True):
        return {
            "completed": False,
            "move": res['move'],
            "valid": valid_res
        }

    board_gpt = push_move(board=copy.deepcopy(
        board), move=chess.Move.from_uci(res['move']))

    return {
        "completed": board_gpt.is_checkmate(),
        "move": res['move'],
        "valid": valid_res
    }


In [11]:
context = np.array([])

# for training-dataset purposes
# for index, row in train_df_one.iterrows():
#     context = np.append(context, process_training_row(row))

messages = np.array([
    {
        "role": "system",
        "content": f"""      
                You are a chess engine playing that solves chess puzzles. \
                Your main task will be to solve chess puzzles.
                
                Therefore, I will provide you with board position in \
                'FEN (Forsyth–Edwards Notation)' format. Also all the \
                legal moves available for that FEN board in UCI (Universal \
                Chess Interface) format.
                
                The prompt input structure will be something like this:
                
                ```
                    "fen": <fen>
                    "valid_moves" : <valid_moves>
                ```
                
                Your task is to choose one of the moves from the legal \
                moves list provided in the prompt that produces a mate \ 
                in one to the provided FEN
                                
                Please do not provide any extra definition or information, just output \
                the movement in a JSON Object with the following format:
                
                ```
                    "move": <your_move>
                ```
            """,
    },
])

context = np.append(messages, context)


In [35]:
filename = build_filename(
    template=TEMPLATE_FILE_PATH,
    temperature=TEMPERATURE,
    dir=OUTPUT_DIR,
    model=MODEL,
    ext=EXTENSION_FILE
)

error_filename = build_filename(
    template=TEMPLATE_ERROR_FILE_PATH,
    temperature=TEMPERATURE,
    dir=OUTPUT_ERROR_DIR,
    model=MODEL,
)

print(f"[exp-path] : {filename}")
print(f"[error-exp-path] : {error_filename}")

_df = pd.DataFrame(columns=build_columns(VARIABILITY_TRIES))

try:
    FILE = open(filename, 'w')
    for index, row in checkmate_df.iterrows():
        try:
            # only first 100 rows [testing]
            if (index == MAX_PUZZLES):
                break

            # list to store results
            results = []
            final_column = []
            # store puzzle id
            results.append(row['PuzzleId'])
            # store dataset last move
            results.append(row['Moves'].split().pop())

            # 3 iterations for variability purposes
            for i in range(VARIABILITY_TRIES):
                result = process_test_row(
                    row=row, context=context, debug=False)
                results.append(result['move'])
                results.append(result['valid'])
                results.append(result['completed'])
                final_column.append(result['completed'])

                if ("gpt-4" in MODEL):
                    time.sleep(REQ_INTERVAL)

            results.append(' '.join(str(e) for e in final_column))

            if (DEBUG is True):
                print(f"[{row['PuzzleId']}]: { results }")

            _df.loc[len(_df)] = results
        except Exception as e:                    
            if (DEBUG is True):
                print(f"[{row['PuzzleId']}]: { e }")
            
            ERROR_FILE = open(error_filename, 'w')
            ERROR_FILE.write(f"[{row['PuzzleId']}]: { e }\n")
            ERROR_FILE.close()

    print(f"Finished experiment and stored at {filename}")
except Exception as e:
    if (DEBUG is True):
        print(f"{ e }\n")
        
_df.to_csv(filename, index=False)



[exp-path] : ./out/26_07-data2-gpt-3.5-turbo-16k-0_1.csv
[error-exp-path] : ./out/error/26_07-error-gpt-3.5-turbo-16k-0.txt
[001gi]: ['001gi', 'a5c3', 'h8g8', True, False, 'h8g8', True, False, 'h8g8', True, False, 'False False False']
[001wb]: ['001wb', 'g6h5', 'h8g8', True, False, 'h8g8', True, False, 'h8g8', True, False, 'False False False']
[002CP]: ['002CP', 'g6c2', 'g6f7', True, False, 'g6f7', True, False, 'g6f7', True, False, 'False False False']
[003IX]: ['003IX', 'e4g3', 'e7f8', True, False, 'e7f8', True, False, 'e7f8', True, False, 'False False False']
[004iZ]: ['004iZ', 'g3g7', 'd4g7', True, False, 'd4g7', True, False, 'd4g7', True, False, 'False False False']
Finished experiment and stored at ./out/26_07-data2-gpt-3.5-turbo-16k-0_1.csv
